In [ ]:
import requests
import folium
from folium import Circle
from folium import Marker
import json
import random
from geopy.distance import geodesic

In [ ]:
# Charger les POI depuis le fichier JSON
with open("poi_categories.json", "r") as f:
    POI = json.load(f)

def get_poi_context(poi_element, mode="similarity"):
    amenity = POI["amenity"]

    # Trouver la sous-catégorie contenant l'élément choisi
    subcategory = None
    for cat, elements in amenity.items():
        if poi_element in elements:
            subcategory = cat
            break

    if not subcategory:
        return -1

    if mode == "similarity":
        # Retourner les éléments de la même sous-catégorie, sauf celui choisi
        neighbors = [e for e in amenity[subcategory] if e != poi_element]
        return {
            "mode": "similarity",
            "input_poi": poi_element,
            "subcategory": subcategory,
            "related_pois": neighbors
        }

    elif mode == "diversity":
        # Choisir une autre sous-catégorie pour la diversité
        other_cats = [cat for cat in amenity if cat != subcategory]
        diversity_cat = random.choice(other_cats)
        diversity_elements = amenity[diversity_cat]
        return {
            "mode": "diversity",
            "input_poi": poi_element,
            "subcategory": subcategory,
            "diversity_cat": diversity_cat,
            "related_pois": diversity_elements
        }

    else:
        return "Mode invalide. Utilise 'similarity' ou 'diversity'."


In [ ]:
def query_pois_from_google(center, radius, api_key):
    lat, lon = center
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "key": api_key
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    
    pois = []
    for result in results:
        pois.append({
            "lat": result["geometry"]["location"]["lat"],
            "lon": result["geometry"]["location"]["lng"],
            "semantic": result.get("types", [None])[0]  # Récupérer le type principal
        })
    return pois

In [ ]:
def query_pois_from_osm(center, radius):
    """
    Interroge l'API Overpass pour récupérer des POI autour d'une localisation donnée.

    :param center: Tuple (latitude, longitude) du centre de recherche.
    :param radius: Rayon de recherche en mètres.
    :return: Liste de POI avec latitude, longitude et tags OSM.
    """
    lat, lon = center
    url = "https://overpass-api.de/api/interpreter"
    headers = {'User-Agent': 'TrainingLocPrivacy/1.0 ()'}
    
    # Construire la requête Overpass
    query = f"""
    [out:json];
    (
      node(around:{radius},{lat},{lon})["amenity"];
      node(around:{radius},{lat},{lon})["shop"];
      node(around:{radius},{lat},{lon})["tourism"];
    );
    out body;
    """
    
    # Envoyer la requête
    response = requests.post(url, headers=headers, data={"data": query})
    response.raise_for_status()
    results = response.json().get("elements", [])
    
    # Traiter les résultats pour extraire les POI
    pois = []
    for result in results:
        pois.append({
            "lat": result.get("lat"),
            "lon": result.get("lon"),
            "semantic": result.get("tags", {}).get("amenity") or  # Priorité à "amenity"
                        result.get("tags", {}).get("shop") or    # Sinon "shop"
                        result.get("tags", {}).get("tourism")   # Sinon "tourism"
        })
    
    return pois

In [ ]:
def visualize_radius_and_pois(center, radius, pois):
    """
    Visualise un rayon et les POIs sur une carte.
    
    :param center: Tuple (latitude, longitude) du centre.
    :param radius: Rayon en mètres.
    :param pois: Liste de POIs sous la forme [(lat, lon, semantic)].
    :return: Carte Folium interactive.
    """
    lat, lon = center

    # Créer la carte centrée sur le point donné
    map_center = folium.Map(location=[lat, lon], zoom_start=15)

    # Ajouter le cercle représentant le rayon
    Circle(
        location=center,
        radius=radius,  # Rayon en mètres
        color='blue',
        fill=True,
        fill_opacity=0.2
    ).add_to(map_center)

    # Ajouter le marqueur du centre
    Marker(
        location=center,
        popup="Centre",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map_center)

    # Ajouter les POIs
    for poi in pois:
        poi_lat = poi['lat']
        poi_lon = poi['lon']
        semantic = poi['semantic']
        Marker(
            location=[poi_lat, poi_lon],
            popup=f"Semantic: {semantic}, Lat: {poi_lat}, Lon: {poi_lon}",
            icon=folium.Icon(color="green", icon="ok-sign")
        ).add_to(map_center)

    return map_center

In [ ]:
def filter_pois_by_min_distance(pois, min_distance):
    selected = []
    for poi in pois:
        too_close = False
        for other in selected:
            dist = geodesic((poi["lat"], poi["lon"]), (other["lat"], other["lon"])).meters
            if dist < min_distance:
                too_close = True
                break
        if not too_close:
            selected.append(poi)
    return selected

In [ ]:
def analyze_and_choose_random_node(file_path):
    """
    Analyse un fichier JSON contenant un graphe réduit et choisit un nœud au hasard.
    
    :param file_path: Chemin vers le fichier JSON.
    :return: Nœud sélectionné au hasard.
    """
    # Lire le contenu du fichier JSON
    with open(file_path, 'r') as file:
        graph_data = json.load(file)

    # Extraire les nœuds du graphe
    nodes = graph_data.get('nodes', [])

    # Vérifier s'il y a des nœuds dans le graphe
    if not nodes:
        print("Le graphe ne contient aucun nœud.")
        return None
    else:
        # Sélectionner un nœud au hasard
        random_node = random.choice(nodes)
            # print("Nœud sélectionné au hasard :")
             #print(json.dumps(random_node, indent=4))
        return random_node

In [ ]:
def find_valid_grid(center, R_min, R_max, delta_R, min_poi, min_diversity,
                    valid_semantics, min_distance_between_pois=0,
                    api_key=None, api_type="google"):
    """
    Trouver une grille valide en ajustant dynamiquement le rayon autour d'une localisation donnée,
    avec une distance minimale entre POIs (p3).
    """
    R = R_min
    while R <= R_max:
        print(f"Recherche avec un rayon de {R} mètres...")

        # Récupération des POIs selon l'API
        if api_type == "google":
            pois = query_pois_from_google(center, R, api_key)
        elif api_type == "osm":
            pois = query_pois_from_osm(center, R)
        else:
            raise ValueError("API non supportée. Utilisez 'google' ou 'osm'.")

        # Filtrer par sémantiques valides
        filtered_pois = [poi for poi in pois if poi['semantic'] in valid_semantics]

        # Appliquer la distance minimale entre POIs (si > 0)
        if min_distance_between_pois > 0:
            filtered_pois = filter_pois_by_min_distance(filtered_pois, min_distance_between_pois)

        # Vérifier diversité et multiplicité
        unique_semantics = set(poi['semantic'] for poi in filtered_pois)
        print(f"{len(filtered_pois)} POI trouvés, {len(unique_semantics)} sémantiques uniques.")

        if len(filtered_pois) >= min_poi and len(unique_semantics) >= min_diversity:
            print("Grille valide trouvée.")
            map_result = visualize_radius_and_pois(center, R, filtered_pois)
            map_result.save("radius_map.html")
            return filtered_pois

        R += delta_R

    print("Aucune grille valide trouvée après avoir atteint le rayon maximum.")
    return {
        "error": "Aucune grille valide trouvée",
        "last_radius": R - delta_R,
        "pois_checked": pois,
        "Rayon": R
    }


In [ ]:
file_path = "Generated_Data/Data_For_2025-03-17_16-30-44/user_graph_33a441b7-3d2b-4856-96dd-35f08acff4fc_from_Strasbourg.json"
node = analyze_and_choose_random_node(file_path)
random_node_position = (node["position"][1], node["position"][0])
res_sim = get_poi_context(node["category"], mode="diversity")
while res_sim == -1:
    node = analyze_and_choose_random_node(file_path)
    random_node_position = (node["position"][1], node["position"][0])
    res_sim = get_poi_context(node["category"], mode="diversity")
print(res_sim.get("related_pois"))
print(random_node_position)
R_min = 10  # Rayon initial (en mètres)
R_max = 20000  # Rayon maximal (en mètres)
delta_R = 100  # Incrément du rayon
min_poi = 10 # Minimum de POI requis
min_diversity = 5  # Minimum de sémantiques distinctes
valid_semantics = res_sim.get("related_pois")  # Sémantiques valides
center = random_node_position  # Position du nœud sélectionné
min_distance_between_pois = 100  # Distance minimale entre POIs

# Pour OpenStreetMap
result_osm = find_valid_grid(center, R_min, R_max, delta_R, min_poi, min_diversity, valid_semantics,min_distance_between_pois, api_type="osm")